# Suite: Double Entry
Using Library Python Selenium to test the web site

In [ ]:
# ####### DEBUG ONLY #######
# ####### Load the "autoreload" extension #######
%load_ext autoreload
# ####### always reload modules marked with "%import" #######
%autoreload 2

import os
import sys

sys.path.append("../config")
sys.path.append("../util")
sys.path.append("../tests")

In [ ]:
# Importing packages
from config import config
from datetime import datetime, timedelta
import json
import logging
import logging.config
import pathlib
from pprint import pprint
from web_driver import WebDriver
import time
import fx_test
from fx_test import TestEngine

from selenium.common.exceptions import StaleElementReferenceException, TimeoutException, ElementClickInterceptedException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.common.keys import Keys

import unittest
from ui_helper import UIHelper

## Variables Declaration

In [ ]:
log_config = "../config/logging.conf"

# # Create folder "logs"
# pathlib.Path('../logs').mkdir(exist_ok=True)  
# logging.log_file = datetime.now().strftime('../logs/log_%Y%m%d_%H%M%S.log')
logging.config.fileConfig(log_config, disable_existing_loggers=False)
logger = logging.getLogger(__name__)

suite_name = "suite_double_entry"
suite_case = "suite_01"
logger.info("Test Suite: {}".format(suite_name))
logger.info("Suite Case: {}".format(suite_case))

## Functions Definition

### Main Function Begin
def main(args):

In [ ]:
mode = config["web"]["mode"]
base_url = config["web"]["base_url"]
timeout = config["web"]["timeout"]
user_name = config["web"]["user_name"]
password = config["web"]["password"]

if mode == "windows":
    driver_path = "../../../library/drivers/chrome/win/chromedriver.exe"
else:
    driver_path = "../../../library/drivers/chrome/mac/chromedriver"
    
logger.info("Mode: {}".format(mode))
logger.info("Base Url: {}".format(base_url))
logger.info("User Name: {}".format(user_name))

In [ ]:
# Get web page
instance = TestEngine(mode="windows", driver_path="../../../library/drivers/chrome/win/chromedriver.exe", timeout=30)
instance.get_driver().get(base_url)

<a id='top'></a>
## Table of contents
- <a href='#data_load'>Load Test Data</a>
- <a href='#login'>Login</a>
- <a href='#switch_company'>Switch Company</a>
- <a href='#chart_of_accounts'>Chart of Accounts</a>
- <a href='#journal_entry'>Journal Entry</a>

<a id='login'></a>
### Login
<a href='#top'>top</a>

In [ ]:
ui = UIHelper(instance, user_name=user_name, password=password)
ui.login()

<a id='data_load'></a>
### Load Test Data
<a href='#top'>top</a>

In [ ]:
# Load sample data
data_path =  f"../data/{suite_case}/double_entry.json"
data = ui.load_data(data_path)
    
company_name = data["company_short_name"]
chart_of_accounts = data["chart_of_accounts"]
journal_entries = data["journal_entries"]

<a id='switch_company'></a>
### Switch Company
<a href='#top'>top</a>

In [ ]:
ui.switch_company(company_name)

<a id='chart_of_accounts'></a>
### Chart of Accounts
<a href='#top'>top</a>

In [ ]:
ui.click_side_menu("Double-Entry", "Chart of Accounts")

In [ ]:
for account in chart_of_accounts:
    locator = (By.XPATH, f'.//tbody/tr[contains(@class, "row") and ./td[1 and text()="{account["code"]}"]]')
    elements = instance.find_elements(locator)

    if len(elements) > 0:
        logger.info(f"Found: {account['code']}")
        # scroll to the element 
        instance.scroll_to_element(elements[0])
        ui.click_edit(elements[0])
    else:
        logger.info(f"Not Found: {account['code']}")
        ui.click_add()
        
    ui.enter_text((By.ID, "name"), account["name"])
    ui.enter_text((By.ID, "code"), account["code"])
    ui.select_dropdown("Type", account["type"])
    ui.enter_text((By.ID, "description"), account["description"])
    instance.scroll_down()
    ui.select_toggle_button("Enabled", account["enabled"])
    
    ui.click_save()
    instance.wait(5)

<a id='journal_entry'></a>
### Journal Entry
<a href='#top'>top</a>

In [ ]:
ui.click_side_menu("Double-Entry", "Journal Entry")

In [ ]:
for journal in journal_entries:
    #select "Add New" button
    ui.click_add()
    ui.select_date_picker("Date", journal["date"])
    ui.enter_text((By.ID, "reference"), journal["reference"])
    ui.enter_text((By.ID, "description"), journal["description"])
    instance.scroll_down()
    # Click "add" button for new items
    instance.click_element((By.XPATH, './/button[@id="button-add-item"]'))
    
    locator = (By.XPATH, '//div[@class="card-body"]//div[contains(@class, "col-md-12") and .//*[text()[contains(., "Items")]]]')
    elements = instance.find_elements(locator)
    # to make Items visible
    instance.scroll_to_element(elements[0])

    for i in range(len(journal["items"])):
        account_type = journal["items"][i]["account"]
        instance.click_element((By.XPATH, f'.//tbody/tr[{i+1}]//span[@class="el-input__prefix"]'))
        xpath=f'.//div[@class="el-select-dropdown el-popper" and not(contains(@style, "display: none"))]//li[2]//*[text()[contains(., "{account_type}")]]'
        instance.click_element((By.XPATH, xpath))
        if "debit" in journal["items"][i]:
            keyword = "debit"
        else:
            keyword = "credit"
        if "." in str(journal["items"][i][keyword]):
            value = str(journal["items"][i][keyword]) + "000"
        else:
            value = str(journal["items"][i][keyword]) + "0000"
        ui.enter_text((By.XPATH, f'.//tbody/tr[{i+1}]//input[@name="{keyword}"]'), value)
        # add new item
        instance.click_element((By.ID, "button-add-item"))
        instance.scroll_by(0, 50)
    instance.scroll_down()
    ui.click_save()
    instance.wait(5)
    break

In [ ]:
instance.close()